ORDER placing not sucessful

In [ ]:
%run Ib_order.py

In [ ]:
# LOADing the ib.ext and ib.opt Libraries
from time import sleep
from ib.opt import Connection, message
from ib.ext.Contract import Contract
from ib.ext.Order import Order

In [ ]:
#a basic function to capture error messages
def error_handler(msg):
    print "IB: Server Error = ", msg

In [ ]:
# DEFINE a basic function to capture all server replies
def all_replies_handler(msg):
    print "IB: Server Response = ", msg.typeName, msg

In [ ]:
def make_contract(symbol, sec_type, exch, prim_exch, curr):
    """
    symbol - The ticker symbol for the contract
    sec_type - The security type for the contract ('STK' is 'stock')
    exch - The exchange to carry out the contract on
    prim_exch - The primary exchange to carry out the contract on
    curr - The currency in which to purchase the contract"""
    #Contract =Contract()
    Contract.m_symbol = symbol
    Contract.m_secType = sec_type
    Contract.m_exchange = exch
    Contract.m_primaryExch = prim_exch
    Contract.m_currency = curr
    return Contract

In [ ]:
def make_order(action,quantity, price = None):
    """Create an Order object (Market/Limit) to go long/short.

    order_type - 'MKT', 'LMT' for Market or Limit orders
    quantity - Integral number of assets to order
    action - 'BUY' or 'SELL'"""

    if price is not None:
        order = Order()
        order.m_orderType = 'LMT'
        order.m_totalQuantity = quantity
        order.m_action = action
        order.m_lmtPrice = price

    else:
        order = Order()
        order.m_orderType = 'MKT'
        order.m_totalQuantity = quantity
        order.m_action = action


    return order

In [ ]:
cid = 303

In [ ]:
# Main code
#con = ibConnection()
con = Connection.create(port=7496, clientId=123)# this also workable
con.connect()
con.register(my_account_handler, 'UpdateAccountValue')

#con.register(my_tick_handler, message.tickSize, message.tickPrice)
con.register(error_handler, 'Error')
con.registerAll(all_replies_handler)
    # Map server replies to "print_portfolio_position" function for "UpdatePortfolio" client requests
#con.register(print_portfolio_position, 'UpdatePortfolio')
#con.register(reply_handler, 'UpdateAccountValue')
    # Make client request for AccountUpdates (includes request for Portfolio positions)

#con.reqAccountUpdates(1, '')
#con.reqMktData(1,Contract, '', False)

In [ ]:
con.enableLogging()

In [ ]:
#tws.reqOpenOrders()
con.reqAllOpenOrders()

In [ ]:
    # Make client request for AccountUpdates (includes request for Portfolio positions)
#con.reqAccountUpdates(1, '')
    # Stop client request for AccountUpdates
#con.reqAccountUpdates(0, '')

In [ ]:
sleep(5)
oid = cid
cont = make_contract('XLU', 'STK', 'SMART', 'SMART', 'USD')
offer = make_order('BUY', 1, 50.14)
#offer = make_order('BUY', 1,'MKT')
con.placeOrder(oid, cont, offer)
con.disconnect()
#x = raw_input('enter to resend')
cid += 1

In [ ]:
con.reconnect()

In [ ]:
# Create a contract object
# Update the parameters to be sent to the Market Order request
order_ticker = Contract()
order_ticker.m_symbol = 'QQQ'
order_ticker.m_secType = 'STK'
order_ticker.m_exchange = 'SMART'
order_ticker.m_primaryExch = 'SMART'
order_ticker.m_currency = 'USD'
order_ticker.m_localSymbol = 'QQQ'

In [ ]:
# Create an order object
# Update the parameters to be sent to the Market Order request
order_desc = Order()
order_desc.m_minQty = 100
order_desc.m_lmtPrice = 117.69
order_desc.m_orderType = 'LMT'
order_desc.m_totalQuantity = 100
order_desc.m_action = 'SELL'

In [ ]:
# Send the Market Order request
con.placeOrder(cid, order_ticker, order_desc)
print 'disconnected',con.disconnect()
cid +=1

In [ ]:
from ib.opt import ibConnection, message
from ib.ext.Contract import Contract
from time import sleep

In [ ]:
def my_callback_handler(msg):
    inside_mkt_bid = ''
    inside_mkt_ask = ''

    if msg.field == 1:
        inside_mkt_bid = msg.price
        print 'bid', inside_mkt_bid
    elif msg.field == 2:
        inside_mkt_ask = msg.price
        print 'ask', inside_mkt_ask

In [ ]:
tws = ibConnection()
tws.register(my_callback_handler, message.tickSize, message.tickPrice)
tws.connect()

In [ ]:
c = Contract()
c.m_symbol = "QQQ"
c.m_secType = "STK"
c.m_exchange = "SMART"
c.m_currency = "USD"

In [ ]:
tws.reqMktData(1,c,"",False)
sleep(4)

In [ ]:
print 'All done'
tws.disconnect()

ANOTHER 

http://stackoverflow.com/questions/30130319/ibpy-extract-api-responces-for-multiple-contracts

In [ ]:
from __future__ import print_function #I'm using 3.x style print
import pandas as pd
from ib.opt import ibConnection, message
from ib.ext.Contract import Contract
from ib.ext.TickType import TickType as tt
from time import sleep

In [ ]:
#better to read these from a file
contracts = pd.DataFrame([
        ['IBM','SMART','USD'],
        ['AAPL','SMART','USD'],
        ['GOOG','SMART','USD'],
        ['QQQ','SMART','USD'],
        ['SPY','SMART','USD','201506','1000']
])

# make decent column names
contracts.columns = ['sym','exch','curr','expiry','mult']
#add these specific column names to match the name returned by TickType.getField()
contracts['bidPrice'] = 0
contracts['askPrice'] = 0
contracts['lastPrice'] = 0

In [ ]:
def error_handler(msg):
    print (msg)

In [ ]:
def my_callback_handler(msg):
    if msg.field in [tt.BID,tt.ASK,tt.LAST]:
        #now we can just store the response in the data frame
        contracts.loc[msg.tickerId,tt.getField(msg.field)] = msg.price
        if msg.field == tt.LAST:
            print(contracts.loc[msg.tickerId,'sym'],msg.price)

In [ ]:
tws = ibConnection(clientId=1)
tws.register(my_callback_handler, message.tickPrice, message.tickSize)
tws.register(error_handler, 'Error')
tws.connect()

In [ ]:
for index, row in contracts.iterrows():
    c = Contract()
    c.m_symbol = row['sym']
    c.m_exchange = row['exch']
    c.m_currency = row['curr']
    c.m_secType = 'STK' if row['expiry'] is None else 'FUT'
    c.m_expiry = row['expiry']
    c.m_multiplier = row['mult']
    # the tickerId is just the index in but for some reason it needs str()
    tws.reqMktData(str(index),c,"",False)

In [ ]:
tws.disconnect()

In [ ]:
ANOTHER

In [1]:
from ib.ext.Contract import Contract
from ib.opt import ibConnection, message
from time import sleep

In [2]:
# print all messages from TWS
def watcher(msgs):
    print msgs

In [3]:
# show Bid and Ask quotes
def my_BidAsk(msg):
    if msg.field == 1:
        print '%s:%s: bid: %s' % (contractTuple[0],
                       contractTuple[6], msg.price)
    elif msg.field == 2:
        print '%s:%s: ask: %s' % (contractTuple[0], contractTuple[6], msg.price)

def makeStkContract(contractTuple):
    newContract = Contract()
    newContract.m_symbol = contractTuple[0]
    newContract.m_secType = contractTuple[1]
    newContract.m_exchange = contractTuple[2]
    newContract.m_currency = contractTuple[3]
    newContract.m_expiry = contractTuple[4]
    newContract.m_strike = contractTuple[5]
    newContract.m_right = contractTuple[6]
    print 'Contract Values:%s,%s,%s,%s,%s,%s,%s:' % contractTuple
    return newContract

In [4]:
%run ibpy_marketdata.py

Server Version: 76
TWS Time at connection:20160919 18:55:02 IST
<managedAccounts accountsList=U1329952>
<nextValidId orderId=14648>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfarm.us>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:hfarm>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfuture>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usbond>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:jfarm>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:eufarm>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usopt>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfarm>
<error id=-1, errorCode=2106, errorMsg=HMDS data farm connection is OK:ilhmds>
<error id=-1, errorCode=2106, errorMsg=HMDS data farm co

In [5]:
%run api_coverage.py

Server Version: 76
TWS Time at connection:20160919 18:55:20 IST

Seen Message Types (count):
    AccountDownloadEnd (1)
    AccountSummary (2)
    AccountSummaryEnd (1)
    ContractDetails (1)
    ContractDetailsEnd (1)
    CurrentTime (1)
    Error (22)
    ExecDetailsEnd (2)
    ManagedAccounts (2)
    NextValidId (2)
    OpenOrderEnd (2)
    Position (4)
    PositionEnd (1)
    RealtimeBar (1)
    ScannerParameters (1)
    TickPrice (8)
    TickSize (9)
    TickString (1)
    UpdateAccountTime (7)
    UpdateAccountValue (127)
    UpdatePortfolio (4)

Unseen Message Types (help):
    BondContractDetails
    CommissionReport
    ConnectionClosed (may work if TWS is closed during script)
    DeltaNeutralValidation
    ExecDetails
    FundamentalData
    HistoricalData
    MarketDataType
    OpenOrder (only works with existing order(s) before connecting)
    OrderStatus
    ReceiveFA (does not work with edemo account)
    ScannerData
    ScannerDataEnd
    TickEFP
    TickGeneric
    Ti